In [1]:
from PIL import Image 
from IPython.display import display 
import random
import json
from functools import reduce
import os
import glob

In [2]:
# Load config file
layer_config = {}
try:
    with open('layer_config.json', 'r') as json_file:
        layer_config = json.load(json_file)
except FileNotFoundError:
    raise Exception('layer_config.json is not found.')

In [3]:
print(layer_config)

{'layers': {'Hand': {'traits': ['Hand_13.png', 'Hand_07.png', 'Hand_06.png', 'Hand_12.png', 'Hand_04.png', 'Hand_10.png', 'Hand_11.png', 'Hand_05.png', 'Hand_01.png', 'Hand_15.png', 'Hand_14.png', 'Hand_16.png', 'Hand_02.png', 'Hand_03.png', 'Hand_17.png', 'Hand_19.png', 'Hand_18.png', 'Hand_20.png', 'Hand_08.png', 'Hand_09.png', 'Hand_21.png'], 'weights': [4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904762, 4.761904761904788]}, 'Eye': {'traits': ['Eye_01.png', 'Eye_02.png', 'Eye_03.png', 'Eye_07.png', 'Eye_06.png', 'Eye_04.png', 'Eye_05.png', 'Eye_08.png', 'Eye_09.png'], 'weights': [11.11111111111111, 11.11111111111111, 11.11111111111111, 11.11111111111111, 11.11

In [4]:
## Generate Traits

TOTAL_IMAGES = 100 # Number of random unique images we want to generate

all_images = []

layers = []

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings
    for layer in layer_config["layers"]:
        new_image[layer] = random.choices(layer_config["layers"][layer]['traits'], layer_config["layers"][layer]['weights'])[0]
    
    # Create new image if has incampatable trait
    for incompat in layer_config["incompatibilities"]:
        for attr in new_image:
            if new_image[incompat['layer']] == incompat['value'] and new_image[attr] in incompat['incompatible_with']:
                return create_new_image()
        
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)
    


In [5]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [6]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [7]:
print(all_images)

[{'Hand': 'Hand_18.png', 'Eye': 'Eye_06.png', 'Body': 'Body_6.png', 'Mouth': 'Mouth_02.png', 'Cheek': 'Cheek_02.png', 'Head': 'Head_09.png', 'BG': 'BG_05.png', 'Cloth': 'Cloth_20.png', 'tokenId': 0}, {'Hand': 'Hand_15.png', 'Eye': 'Eye_04.png', 'Body': 'Body_11.png', 'Mouth': 'Mouth_17.png', 'Cheek': 'Cheek_05.png', 'Head': 'Head_13.png', 'BG': 'BG_13.png', 'Cloth': 'Cloth_07.png', 'tokenId': 1}, {'Hand': 'Hand_13.png', 'Eye': 'Eye_03.png', 'Body': 'Body_11.png', 'Mouth': 'Mouth_02.png', 'Cheek': 'Cheek_02.png', 'Head': 'Head_12.png', 'BG': 'BG_14.png', 'Cloth': 'Cloth_10.png', 'tokenId': 2}, {'Hand': 'Hand_13.png', 'Eye': 'Eye_08.png', 'Body': 'Body_4.png', 'Mouth': 'Mouth_09.png', 'Cheek': 'Cheek_03.png', 'Head': 'Head_12.png', 'BG': 'BG_13.png', 'Cloth': 'Cloth_03.png', 'tokenId': 3}, {'Hand': 'Hand_04.png', 'Eye': 'Eye_04.png', 'Body': 'Body_5.png', 'Mouth': 'Mouth_05.png', 'Cheek': 'Cheek_03.png', 'Head': 'Head_03.png', 'BG': 'BG_13.png', 'Cloth': 'Cloth_15.png', 'tokenId': 4}, {'

In [8]:
### Clear metadata and images folder
image_files = glob.glob('./images/*.png')
metadata_files = glob.glob('./metadata/*.json')
files = image_files + metadata_files
for f in files:
    try:
        os.unlink(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

In [9]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [10]:
#### Generate Images 
for items in all_images:
    images = [Image.open(f'./trait-layers/{layer}/{items[layer]}').convert('RGBA') for layer in layer_config["layer_order"] if layer != 'tokenId']
    
    # Create each composite
    comp = reduce(lambda pre, cur: Image.alpha_composite(pre, cur), images)
    
    #Convert to RGB
    rgb_im = comp.convert('RGB')
    file_name = str(items["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)

In [11]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)

IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for items in data:
    token_id = items['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": [items[layer] for layer in items if layer != 'tokenId']
    }

    with open(f'./metadata/{str(token_id)}.json', 'w') as outfile:
        json.dump(token, outfile, indent=4)
        
f.close()